In [36]:
import pandas as pd
import numpy as np
import matplotlib as plt

<h2>Clean Parking Meters Location Data</h2>
<p>Just selected and saved columns for Pole ID, Longitude, and Latitude data</p>

In [37]:
meters_df = pd.read_csv('raw_parking_meter_location.csv')
meters_loc_df = meters[['pole','longitude','latitude']]

#Cleaned traffic meter location data with just latitude and longitude
meters_loc_df.to_csv('clean_parking_meter_location.csv')

In [38]:
meters_df

,zone,area,sub_area,pole,config_id,config_name,longitude,latitude
0,City,Barrio Logan,1000 CESAR CHAVEZ WAY,CC-1003,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.145178,32.700353
1,City,Barrio Logan,1000 CESAR CHAVEZ WAY,CC-1005,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.145178,32.700352
2,City,Barrio Logan,1000 CESAR CHAVEZ WAY,CC-1011,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.145349,32.700155
3,City,Barrio Logan,1000 CESAR CHAVEZ WAY,CC-1013,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.145405,32.700107
4,City,Barrio Logan,1000 CESAR CHAVEZ WAY,CC-1015,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.145539,32.699987
5,City,Barrio Logan,1000 CESAR CHAVEZ WAY,CC-1017,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.145540,32.699985
6,City,Barrio Logan,1000 CESAR CHAVEZ WAY,CC-1019,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.145545,32.699981
7,City,Barrio Logan,1100 CESAR CHAVEZ WAY,CC-1103,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.145973,32.699544
8,City,Barrio Logan,1100 CESAR CHAVEZ WAY,CC-1105,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.145973,32.699545
9,City,Barrio Logan,1100 CESAR CHAVEZ WAY,CC-1107,9000,2 Hour Max $1.25 HR 8am-6pm Mon-Sat,-117.146065,32.699490
